## Working notebook for Python data analysis project

In [47]:
import numpy as np
import pandas as pd
import re

In [67]:
surveys_dtypes = {'Facility ID':'string',
 'Facility Name':'string',
 'Address':'string',
 'City':'string',
 'State':'string',
 'ZIP Code':'string',
 'County Name':'string',
 'Phone Number':'string',
 'HCAHPS Measure ID':'string',
 'HCAHPS Question':'string',
 'HCAHPS Answer Description':'string',
 'Patient Survey Star Rating':'Int32',
 'Patient Survey Star Rating Footnote':'string',
 'HCAHPS Answer Percent':'Int32',
 'HCAHPS Answer Percent Footnote':'string',
 'HCAHPS Linear Mean Value':'Int32',
 'Number of Completed Surveys':'Int32',
 'Number of Completed Surveys Footnote':'string',
 'Survey Response Rate Percent':'Int32',
 'Survey Response Rate Percent Footnote':'string',
 'Start Date':'string',
 'End Date':'string'}

In [68]:
surveys = pd.read_csv('HCAHPS-Hospital.csv', low_memory = False, 
                      dtype = surveys_dtypes, na_values = ['Not Available', 'Not Applicable'])

In [72]:
unpland_dtypes = {'Facility ID':'string',
 'Facility Name':'string',
 'Address':'string',
 'City':'string',
 'State':'string',
 'ZIP Code':'string',
 'County Name':'string',
 'Phone Number':'string',
 'Measure ID':'string',
 'Measure Name':'string',
 'Compared to National':'string',
 'Denominator':'float',
 'Score':'float',
 'Lower Estimate':'float',
 'Higher Estimate':'float',
 'Number of Patients':'Int32',
 'Number of Patients Returned':'Int32',
 'Footnote':'string',
 'Start Date':'string',
 'End Date':'string'}

In [73]:
unpland = pd.read_csv('Unplanned_Hospital_Visits-Hospital.csv', 
                      low_memory = False, 
                      dtype = unpland_dtypes, na_values = ['Not Available', 'Not Applicable'])

In [15]:
print(surveys.shape)
print(surveys['Facility ID'].unique().size)
print(surveys['Start Date'].unique().size)
print(surveys['End Date'].unique().size)
print(surveys['Start Date'].unique()[0])
print(surveys['End Date'].unique()[0])

(452538, 22)
4866
1
1
01/01/2019
12/31/2019


In [50]:
print(unpland.shape)
print(unpland['Facility ID'].unique().size)
print(unpland['Start Date'].unique().size)
print(unpland['End Date'].unique().size)
print(unpland['Start Date'].unique())
print(unpland['End Date'].unique())

(68124, 20)
4866
4
2
['07/01/2017' '01/01/2017' '01/01/2019' '07/01/2019']
['12/01/2019' '12/24/2019']


#### Note that the survey data is for 2019 only, whereas the unplanned visits data can span periods beginning in 2017. It is impossible (or at least problematic) to extract only 2019 data from unplanned visits because a single measure (single row) can have a start date in 2017 and an end date in 2019. We will continue with the analysis but this is certainly an issue to be aware of and communicated along with the results.

In [18]:
list(enumerate(surveys.columns.tolist()))

[(0, 'Facility ID'),
 (1, 'Facility Name'),
 (2, 'Address'),
 (3, 'City'),
 (4, 'State'),
 (5, 'ZIP Code'),
 (6, 'County Name'),
 (7, 'Phone Number'),
 (8, 'HCAHPS Measure ID'),
 (9, 'HCAHPS Question'),
 (10, 'HCAHPS Answer Description'),
 (11, 'Patient Survey Star Rating'),
 (12, 'Patient Survey Star Rating Footnote'),
 (13, 'HCAHPS Answer Percent'),
 (14, 'HCAHPS Answer Percent Footnote'),
 (15, 'HCAHPS Linear Mean Value'),
 (16, 'Number of Completed Surveys'),
 (17, 'Number of Completed Surveys Footnote'),
 (18, 'Survey Response Rate Percent'),
 (19, 'Survey Response Rate Percent Footnote'),
 (20, 'Start Date'),
 (21, 'End Date')]

In [30]:
surv_red_ = surveys.iloc[:,[0, 8, 11, 13, 15, 16, 18]]

In [31]:
surv_red_.head()

,Facility ID,HCAHPS Measure ID,Patient Survey Star Rating,HCAHPS Answer Percent,HCAHPS Linear Mean Value,Number of Completed Surveys,Survey Response Rate Percent
0,010001,H_COMP_1_A_P,Not Applicable,77,Not Applicable,507,21
1,010001,H_COMP_1_SN_P,Not Applicable,7,Not Applicable,507,21
2,010001,H_COMP_1_U_P,Not Applicable,16,Not Applicable,507,21
3,010001,H_COMP_1_LINEAR_SCORE,Not Applicable,Not Applicable,90,507,21
4,010001,H_COMP_1_STAR_RATING,3,Not Applicable,Not Applicable,507,21


In [54]:
re.search('LINEAR_SCORE$', 'H_COMP_1_LINEAR_SCORE') != None

False

In [41]:
surv_red = surv_red_.astype('string')

In [56]:
list(enumerate(surv_red.columns.tolist()))

[(0, 'Facility ID'),
 (1, 'HCAHPS Measure ID'),
 (2, 'Patient Survey Star Rating'),
 (3, 'HCAHPS Answer Percent'),
 (4, 'HCAHPS Linear Mean Value'),
 (5, 'Number of Completed Surveys'),
 (6, 'Survey Response Rate Percent')]

In [58]:
surv_red['HCAHPS Measure ID'].apply(lambda x: re.search('STAR_RATING$', x) != None)

0         False
1         False
2         False
3         False
4          True
          ...  
452533    False
452534    False
452535    False
452536     True
452537     True
Name: HCAHPS Measure ID, Length: 452538, dtype: bool

In [59]:
star_ratings = surv_red.iloc[:, [0, 1, 2]][surv_red['HCAHPS Measure ID'].apply(lambda x: re.search('STAR_RATING$', x) != None)]

In [60]:
star_ratings

,Facility ID,HCAHPS Measure ID,Patient Survey Star Rating
4,010001,H_COMP_1_STAR_RATING,3
18,010001,H_COMP_2_STAR_RATING,3
32,010001,H_COMP_3_STAR_RATING,3
43,010001,H_COMP_5_STAR_RATING,3
53,010001,H_COMP_6_STAR_RATING,4
...,...,...,...
452521,670285,H_CLEAN_STAR_RATING,Not Available
452526,670285,H_QUIET_STAR_RATING,Not Available
452531,670285,H_HSP_RATING_STAR_RATING,Not Available
452536,670285,H_RECMND_STAR_RATING,Not Available


In [61]:
linear_scores = surv_red.iloc[:, [0, 1, 4]][surv_red['HCAHPS Measure ID'].apply(lambda x: re.search('LINEAR_SCORE$', x) != None)]

In [62]:
linear_scores

,Facility ID,HCAHPS Measure ID,HCAHPS Linear Mean Value
3,010001,H_COMP_1_LINEAR_SCORE,90
17,010001,H_COMP_2_LINEAR_SCORE,91
31,010001,H_COMP_3_LINEAR_SCORE,84
42,010001,H_COMP_5_LINEAR_SCORE,80
52,010001,H_COMP_6_LINEAR_SCORE,90
...,...,...,...
452506,670285,H_COMP_7_LINEAR_SCORE,Not Available
452520,670285,H_CLEAN_LINEAR_SCORE,Not Available
452525,670285,H_QUIET_LINEAR_SCORE,Not Available
452530,670285,H_HSP_RATING_LINEAR_SCORE,Not Available


In [45]:
surv_red.groupby('Facility ID').count()['HCAHPS Measure ID'].unique()

array([93], dtype=int64)

In [46]:
surv_red[surv_red['Facility ID'] == '010001'].to_csv('all_meas.csv')

In [8]:
unpland.columns.tolist()

['Facility ID',
 'Facility Name',
 'Address',
 'City',
 'State',
 'ZIP Code',
 'County Name',
 'Phone Number',
 'Measure ID',
 'Measure Name',
 'Compared to National',
 'Denominator',
 'Score',
 'Lower Estimate',
 'Higher Estimate',
 'Number of Patients',
 'Number of Patients Returned',
 'Footnote',
 'Start Date',
 'End Date']

#### For the most part footnotes explain why a value is missing, or give information on a restriction that applies to a measure. At this time they are deemed unimportant. 